# Advanced Analog Design
## Demo sizing script: Simple OTA

Attention: this script is only for demonstration purposes and the sizing strategy may be absurd!

This jupyter notebook is based on a Matlab script from Prof. Dr. Ir. Pieter Rombouts. I, Senne Vanden Berghe, former follower from this course, decided that the Matlab 'tyranny' must stop. So I made the gavanon Python library so that the new generation of students could use their much more valuable Python skillset.

Suggestions or feedback is always welcome on [Github](https://github.com/S3nn3k3/gavanon/issues).

Let's start by importing the gavanon Python library.

In [22]:
from gavanon import (Transistor, DoubleTransistor, NMOS, PMOS,
                     VoltageSource, CurrentSource,
                     CadenceCell, CadenceLib)
# equivalent to:
# from gavanon import *
from math import pi

Circuit specifications are defined below.

In [23]:
C_load = 1e-12  # load capacitance is 12 pF
settle_time = 20e-9  # settle time is 20 nano seconds

About the Classes NMOS and PMOS:
These classes contain al the needed parameters to make the sizing.

In [24]:
print(f"NMOS class: {PMOS.__doc__}")
print(f"PMOS class: {PMOS.__doc__}")

NMOS class: Contains all needed values of the PMOS transistor in the 0.35µm technology
    Values:
        * KP_n = 50e-6 A/V²
        * n = 1.3
        * VT = 0.7 V
        * VEarly = 8e6 V/m
        * vsat = 8e4 m/s
        * Cox = 6e-3 F/m²
        * Kf = 4e-28 C²/m²
        * CDB0 = 0.5e-9 F/m
        * CSB0 = 0.5e-9 F/m
        * CGD0 = 0.09e-9 F/m
        * CGS0 = 0.1e-9 F/m
    
PMOS class: Contains all needed values of the PMOS transistor in the 0.35µm technology
    Values:
        * KP_n = 50e-6 A/V²
        * n = 1.3
        * VT = 0.7 V
        * VEarly = 8e6 V/m
        * vsat = 8e4 m/s
        * Cox = 6e-3 F/m²
        * Kf = 4e-28 C²/m²
        * CDB0 = 0.5e-9 F/m
        * CSB0 = 0.5e-9 F/m
        * CGD0 = 0.09e-9 F/m
        * CGS0 = 0.1e-9 F/m
    


## Start of the actual sizing

First do a rough sizing (by hand).
This can be the initialisation of a more complex iterative loop.


##### Size differential pair Mdiff

Note: The chosen current in the example below is of course ridiculous. If you do this in your actual project, you will definitely not pass!

In [25]:
Mdiff = DoubleTransistor("nmos")

Mdiff.ID = 5e-3  # Choose current
Mdiff.gmoverid = 10 # Choose gm over ID

# this assignment is redundant since the Transistor class sets the gatelength default to .35µm.
# but in some cases the gatelength can vary of course!
Mdiff.L = 0.35e-6  # Choose L

Mdiff.gm = Mdiff.gmoverid*Mdiff.ID

W_over_L = Mdiff.gm**2/2/Mdiff.ID/NMOS.KP_n  # Piecewise linear model
Mdiff.W = W_over_L*Mdiff.L
# Wow, this looks cumbersome huh?
# Hint: 
# def foo(transistor: Transistor, some_extra_needed_arguments):
#     if transistor.catergory == "nmos":
#         transistor.W = ...
#     else:
#         ...
# It is completely up to you how to handle this!

Mdiff.ng = 10  # Set number of gates

# Visually verify your transistor via:
print(Mdiff)

Mdiff_a,Mdiff_b wtot=514.706u l=0.350u ng=10 ID=5000.000u gmoverid=10.00


##### Size load transistor Mload

In [26]:
Mload = DoubleTransistor("pmos")

Mload.ID = Mdiff.ID  # Same as differntial pair
Mload.gmoverid = 10  # Choose gm over ID
Mload.gm = Mload.gmoverid*Mload.ID

W_over_L = Mload.gm**2/2/Mload.ID/PMOS.KP_n  # Piecewise linear model
Mload.W = W_over_L*Mload.L

Mload.ng = 10  # Set number of gates

# Visually verify your transistor via:
print(Mload)

Mload_a,Mload_b wtot=1750.000u l=0.350u ng=10 ID=5000.000u gmoverid=10.00


##### Size tail transistor Mtail

In [27]:
Mtail = Transistor("nmos")

Mtail.ID = 2*Mdiff.ID  # Double of the differential pair
Mtail.gmoverid = 10  # Choose gm over ID
Mtail.gm = Mtail.gmoverid*Mtail.ID

W_over_L = Mtail.gm**2/2/Mtail.ID/PMOS.KP_n  # Piecewise linear model
Mtail.W = W_over_L*Mtail.L

Mtail.ng = 10  # Set number of gates

# Visually verify your transistor via:
print(Mtail)

Mtail wtot=3500.000u l=0.350u ng=10 ID=10000.000u gmoverid=10.00


##### Size bias transistor Mbias

In [28]:
m = 10  # scale factor 
Mbias = Mtail/m  # Returns a new impedance scaled transistor object!

# Visually verify your transistor via:
print(Mbias)

Mbias wtot=350.000u l=0.350u ng=1 ID=1000.000u gmoverid=10.00


##### Bias current source

In [29]:
Ibias = CurrentSource(Mbias.ID)
print(f"Current from source is {Ibias.I} A")

Current from source is 0 A


### Improved sizing

Now it is time to set up an iterative loop to improve your sizing. For this example it might not be needed, but it would be needed when working on the project.

Some hints:
* Use matplotlib to visualize data. It can give you extra insights in what is going on. It even might help to show plots when asking questions to the supervisors (but make sure the axes are labeled!). Matplotlib can be installed via `pip install matplotlib` or via the magic command (`%pip install matplotlib`) in the cell below. More info [here](https://matplotlib.org/).
* Use the Python control library. Control can be installed via `pip install control` or via the magic command (`%pip install control`) in the cell below. Some interesting functions are `control.tf`, `control.dcgain`, `control.bode_plot` and `control.pzmap`. But feel free to also explore the time domain and other nice features of the library. More info [here](https://python-control.readthedocs.io/).

In [30]:
# uncomment the package you would like to install
# %pip install matplotlib
# %pip install control 

The parameters that need to be tested trough simulation are given below.

In [31]:
simulation_settling_time = 1e-9  # Add appropriate calculation
slew_rate = 1e-9  # Add appropriate calculation

parasitic_pole = 4e9  # Add appropriate calculation
phase_margin = pi*7/8  # Add appropriate calculation

# Add all relevant specs!

## Make skill file for Cadence

Since now all transistors and current sources are defined, we need to export a text file that can be inported in Cadence. How this is done, is shown below.

##### Generate the Cadence cell class objects

Let's start by first creating al the Cadence cells. You should be familiar with it after the first Cadence introduction.
In this example we have three cells:
* Simple_OTA: Contains the OTA circuit
* Simple_OTA_bias: Contains the biasing circuit for the OTA
* Simple_OTA_TB1: Contains the test bench for the OTA

The transistors that are drawn in Cadence in the simple OTA cell are Mdiff, Mload and Mtail. Let's add them.

In [32]:
ota_cell = CadenceCell("Simple_OTA")

# add transistors to cell
ota_cell.add(Mdiff, Mload, Mtail)

Only Mbias is available is the bias cell.

In [33]:
bias_cell = CadenceCell("Simple_OTA_bias")
bias_cell += Mbias

In the test bench cell only the bias current source should be added (in this example).

In [34]:
tb_cell = CadenceCell("Simple_OTA_TB1")
tb_cell += Ibias

Extra: if you want to change values of resistors, capacitors or other elements you can add custom 'skill commands' to a cell. This is shown in the cell below. Normally you should not need it.

In [37]:
foo = CadenceCell("Foo_Cadence_cell")
foo.add_free_entry("Bar : r=420")

#### Generate the Cadence library class object

Now the library is initiated and the cells are added to the library. 

In [35]:
lib = CadenceLib("OefeningenGAvanont")

# add cells to library
lib += ota_cell
lib += bias_cell
lib += tb_cell

## which is equivalent to:
# lib.add(ota_cell, bias_cell, tb_cell)

The last thing that needs to be done is generating the actual skill file. This is done in the cell below.

Note: When you are working on your own laptop it might be handy to mount your personal Ghent University directory to your laptops file system. More info about this can be found on the [DICT website](https://helpdesk.ugent.be/netdisk/en/bestand_mount.php). As seen in the code cell below the skill file will try to save on the home directory of your personal UGhent drive. Cadence is able to acces that specific location.

In [36]:
try:
    lib.export_sizing("u:/home/python_sizing.il") # export to UGhent mount home directory
    print("Saved file to UGhent drive!")
except:
    lib.export_sizing("simple_ota_sizing.il") # export to local directiory
    print("Saved file to local directory!")

Saved file to local directory!
